<a href="https://colab.research.google.com/github/edubarbier/final-ML/blob/main/Trabajo_Final_CNN_Style_Transfer_Barbier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Style Transfer

<img src="https://i0.wp.com/chelseatroy.com/wp-content/uploads/2018/12/neural_style_transfer.png?resize=768%2C311&ssl=1">

La idea de este trabajo final es reproducir el siguiente paper:

https://arxiv.org/pdf/1508.06576.pdf

El objetivo es transferir el estilo de una imagen dada a otra imagen distinta. 

Como hemos visto en clase, las primeras capas de una red convolucional se activan ante la presencia de ciertos patrones vinculados a detalles muy pequeños.

A medida que avanzamos en las distintas capas de una red neuronal convolucional, los filtros se van activando a medida que detectan patrones de formas cada vez mas complejos.

Lo que propone este paper es asignarle a la activación de las primeras capas de una red neuronal convolucional (por ejemplo VGG19) la definición del estilo y a la activación de las últimas capas de la red neuronal convolucional, la definición del contenido.

La idea de este paper es, a partir de dos imágenes (una que aporte el estilo y otra que aporte el contenido) analizar cómo es la activación de las primeras capas para la imagen que aporta el estilo y cómo es la activación de las últimas capas de la red convolucional para la imagen que aporta el contenido. A partir de esto se intentará sintetizar una imagen que active los filtros de las primeras capas que se activaron con la imagen que aporta el estilo y los filtros de las últimas capas que se activaron con la imagen que aporta el contenido.

A este procedimiento se lo denomina neural style transfer.

# En este trabajo se deberá leer el paper mencionado y en base a ello, entender la implementación que se muestra a continuación y contestar preguntas sobre la misma.

# Una metodología posible es hacer una lectura rápida del paper (aunque esto signifique no entender algunos detalles del mismo) y luego ir analizando el código y respondiendo las preguntas. A medida que se planteen las preguntas, volviendo a leer secciones específicas del paper terminará de entender los detalles que pudieran haber quedado pendientes.

Lo primero que haremos es cargar dos imágenes, una que aporte el estilo y otra que aporte el contenido. A tal fin utilizaremos imágenes disponibles en la web.

In [ ]:
# Imagen para estilo
!wget https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg

# Imagen para contenido
!wget https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg

# Creamos el directorio para los archivos de salida
!mkdir /content/output

--2021-01-31 18:43:44--  https://upload.wikimedia.org/wikipedia/commons/5/52/La_noche_estrellada1.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 223725 (218K) [image/jpeg]
Saving to: ‘La_noche_estrellada1.jpg’

La_noche_estrellada 100%[===================>] 218.48K  --.-KB/s    in 0.007s  

2021-01-31 18:43:44 (30.2 MB/s) - ‘La_noche_estrellada1.jpg’ saved [223725/223725]

--2021-01-31 18:43:44--  https://upload.wikimedia.org/wikipedia/commons/thumb/f/f4/Neckarfront_T%C3%BCbingen_Mai_2017.jpg/775px-Neckarfront_T%C3%BCbingen_Mai_2017.jpg
Resolving upload.wikimedia.org (upload.wikimedia.org)... 208.80.154.240, 2620:0:861:ed1a::2:b
Connecting to upload.wikimedia.org (upload.wikimedia.org)|208.80.154.240|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 153015 (149K) [ima

In [ ]:
%tensorflow_version 1.x
from keras.preprocessing.image import load_img, save_img, img_to_array
import numpy as np
from scipy.optimize import fmin_l_bfgs_b
import time
import argparse

from keras.applications import vgg19
from keras import backend as K
from pathlib import Path

TensorFlow 1.x selected.


Using TensorFlow backend.


In [ ]:
# Definimos las imagenes que vamos a utilizar, y el directorio de salida

base_image_path = Path("/content/775px-Neckarfront_Tübingen_Mai_2017.jpg")
style_reference_image_path = Path("/content/La_noche_estrellada1.jpg")
result_prefix = Path("/content/output")
iterations = 100

# 1) En base a lo visto en el paper ¿Qué significan los parámetros definidos en la siguiente celda?

Respuesta:
La Loss propuesta para generar una imagen es una suma ponderada de una loss que busca minimizar las diferencias en estilo y una que busca minimizar las diferencias en contenido. 
La importancia dada entre estilo y contenido se define con 2 factores que multipican cada una de las losses independientes. En este caso el parametro *style_weight* corresponde al peso que se le asigna al término de estilo en la loss mientras que *content_weight* corresponde al peso asignado al contenido.
La Loss resultante queda definida entonces como
*L* = *style_weight* * *style_loss* +  *content_weight* * *content_loss*


In [ ]:
total_variation_weight = 1 
style_weight = 10
content_weight = 1

In [ ]:
# Definimos el tamaño de las imágenes a utilizar
width, height = load_img(base_image_path).size
img_nrows = 400
img_ncols = int(width * img_nrows / height)

In [ ]:
img = load_img(base_image_path, target_size=(img_nrows, img_ncols))
img = img_to_array(img)
img = np.expand_dims(img, axis=0)
img = vgg19.preprocess_input(img)

In [ ]:
x = img.reshape((img_nrows, img_ncols, 3))


# 2) Explicar qué hace la siguiente celda. En especial las últimas dos líneas de la función antes del return. ¿Por qué?

Ayuda: https://keras.io/applications/

Respuesta: 
La celda define la función *preprocess_image* que se encarga de cargar una imagen, resamplearla por interpolación al tamaño deseado (definido por *img_nrows* , *img_ncols*). Posteriormente convierte la imagen resampleada a una np.array 3D de dimensiones *img_nrows* , *img_ncols* , *3* (rgb) para luego agregar una cuarta dimensión (adelante) que corresponde a las distintas imagenes del batch tal que el tensor resultante tiene las siguientes dimensiones: (n° de imagenes en el batch, alto de imagen (filas), ancho de imagen (columnas), numero de canales (3 por ser RGB) ). En este caso por preprocesar una imagen por vez, el numero de imagenes es 1 en el tensor de salida.

Finalmente convierte la imagen de RGB a BGR dado que los pesos cargados corresponden a pesos pre-entreados en un set de imagenes BGR (Imagenet). Adicionalmente, cada canal de color es centrado con respecto a el dataset de ImagNet. 

In [ ]:
def preprocess_image(image_path):
    img = load_img(image_path, target_size=(img_nrows, img_ncols))
    img = img_to_array(img)
    img = np.expand_dims(img, axis=0)
    img = vgg19.preprocess_input(img)
    return img

# 3) Habiendo comprendido lo que hace la celda anterior, explique de manera muy concisa qué hace la siguiente celda. ¿Qué relación tiene con la celda anterior?

Respuesta:
Realiza las operaciones inversas a aquellas realizas en preprocess_image tal que se pueda obtener una imagen RGB de una imagen preprocesada, o del resultado de la red.
La función comienza eliminando la dimension del batch, posteriormente remueve el centrado de los valores de pixel, convierte a RGB inviritnedo el orden de los canales y se asegura que los valores de queden acotados entre 0 y 255.

In [ ]:
def deprocess_image(x):
    x = x.reshape((img_nrows, img_ncols, 3))
    # Remove zero-center by mean pixel
    x[:, :, 0] += 103.939
    x[:, :, 1] += 116.779
    x[:, :, 2] += 123.68
    # 'BGR'->'RGB'
    x = x[:, :, ::-1]
    x = np.clip(x, 0, 255).astype('uint8')
    return x

In [ ]:
# get tensor representations of our images
# K.variable convierte un numpy array en un tensor, para 
base_image = K.variable(preprocess_image(base_image_path))
style_reference_image = K.variable(preprocess_image(style_reference_image_path))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
combination_image = K.placeholder((1, img_nrows, img_ncols, 3))

Aclaración:

La siguiente celda sirve para procesar las tres imagenes (contenido, estilo y salida) en un solo batch.

In [ ]:
# combine the 3 images into a single Keras tensor
input_tensor = K.concatenate([base_image,
                              style_reference_image,
                              combination_image], axis=0)

In [ ]:
# build the VGG19 network with our 3 images as input
# the model will be loaded with pre-trained ImageNet weights
model = vgg19.VGG19(input_tensor=input_tensor,
                    weights='imagenet', include_top=False)
print('Model loaded.')

# get the symbolic outputs of each "key" layer (we gave them unique names).
outputs_dict = dict([(layer.name, layer.output) for layer in model.layers])


80142336/80134624 [==============================] - 1s 0us/step
Model loaded.


# 4) En la siguientes celdas:

- ¿Qué es la matriz de Gram?¿Para qué se usa?
- ¿Por qué se permutan las dimensiones de x?

La matriz de Gram se utiliza para capturar el estilo dentro de la imagen. Cada término de la matriz de Gram corresponde al producto interno entre el resultado de dos filtros de una misma capa de la red. De esta forma la matriz captura la correlaciones espacial de los features identificados por distintos filtros.

El feature de respuesta tiene dimensiones de ancho, alto y cantidad de filtros.La permutación se hace para que el flatten actue sobre las dimensiones del resultado de cada filtro y no sobre los filtros en si. Es decir para lograr que la matriz sea de dimensiones: n° de filtros, ancho * alto de filtro. 
De esta forma, al realizar el producto punto se obtiene la correlación entre todos los puntos espaciales para distintos filtros.

In [ ]:
def gram_matrix(x):
    features = K.batch_flatten(K.permute_dimensions(x, (2, 0, 1)))
    gram = K.dot(features, K.transpose(features))
    return gram

# 5) Losses:

Explicar qué mide cada una de las losses en las siguientes tres celdas.

*style_loss* corresponde, como su nombre lo indica a la loss que busca obtener igual respuesta entre la referencia de estilo y la imagen generada. Esto lo logra minimizando la suma de las diferencias de los elementos de la matriz de Gram (de cada imagen en una capa dada) elevados al cuadrado. El resultado es normalizado en fucnión de la cantidad de elementos y canales de la imagen.


*content_loss* minimiza la diferencia de contenido entre las imagenes de referencia de contenido y la generada. Esto se logra minimizando la diferencia entre las respuestas de ambas imagenes en una capa dada.

*total_variation_loss* busca reducir la diferencias entre pixeles contiguos en la imagen de salida. Pudiendose interpretarse como un factor de detalle o "sharpening" de la imagen.

La loss total se compone como la suma de la style loss calculada en varias capas (y normalizada respecto de la cantidad de capas) pasada con el factor *style_weight* mas la loss de contenido calculada a partir de una unica capa y pesada con el factor *content_weight* y por ultimo se suma la *total_variation_loss* pesada con el factor *total_variation_weight*

In [ ]:
def style_loss(style, combination):
    assert K.ndim(style) == 3
    assert K.ndim(combination) == 3
    S = gram_matrix(style)
    C = gram_matrix(combination)
    channels = 3
    size = img_nrows * img_ncols
    return K.sum(K.square(S - C)) / (4.0 * (channels ** 2) * (size ** 2))

In [ ]:
def content_loss(base, combination):
    return K.sum(K.square(combination - base))


In [ ]:
def total_variation_loss(x):
    assert K.ndim(x) == 4
    a = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, 1:, :img_ncols - 1, :])
    b = K.square(
        x[:, :img_nrows - 1, :img_ncols - 1, :] - x[:, :img_nrows - 1, 1:, :])
    return K.sum(K.pow(a + b, 1.25))


In [ ]:
# Armamos la loss total
loss = K.variable(0.0)
layer_features = outputs_dict['block5_conv2']
base_image_features = layer_features[0, :, :, :]
combination_features = layer_features[2, :, :, :]
loss = loss + content_weight * content_loss(base_image_features,
                                            combination_features)

feature_layers = ['block1_conv1', 'block2_conv1',
                  'block3_conv1', 'block4_conv1',
                  'block5_conv1']
for layer_name in feature_layers:
    layer_features = outputs_dict[layer_name]
    style_reference_features = layer_features[1, :, :, :] 
    combination_features = layer_features[2, :, :, :]
    sl = style_loss(style_reference_features, combination_features)
    loss = loss + (style_weight / len(feature_layers)) * sl
loss = loss + total_variation_weight * total_variation_loss(combination_image)

In [ ]:
grads = K.gradients(loss, combination_image)

outputs = [loss]
if isinstance(grads, (list, tuple)):
    outputs += grads
else:
    outputs.append(grads)

f_outputs = K.function([combination_image], outputs)

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where



# 6) Explique el propósito de las siguientes tres celdas. ¿Qué hace la función fmin_l_bfgs_b? ¿En qué se diferencia con la implementación del paper? ¿Se puede utilizar alguna alternativa?

Respuesta:
La primer celda define la función de evaluación que devuelve no solo los valores de loss sino tambien los gradientes requeridos para minimizar la loss. 
La segunda celda define una clase con las funciones para ser llamadas por el optimizador posterioromente. Adcionalmente se asegura que la función *eval_loss_and_grads* haya sido evaluada antes de devolver los valores de gradientes.

La tercer celda corresponde a la generación de la imagen combinada. A diferencia de la implementación del paper, la imagen inicial adoptada es la referencia de contenido en lugar de una de ruido blanco.
Partiendo de esta imagen, se itera minimizando la loss, modificando progresivamente la imagen original para lograr el estilo de la imagen de referncia.

La minimización de la loss se realiza utilizando la función *fmin_l_bfgs_b*. Partiendo de un valor inicial (tensorial) que corresponde con la imagen base de referencia, usando la función de gradientes provista (definida dentro del Evaluator). La función devuelve una nueva imagen que minimiza la loss tras 20 iteraciones internas.

La minimización se define iterativa, actualizando el valor de x, la imagen, tras cada solución de  *fmin_l_bfgs_b* donde se actualiza el valor "inicial" para el proximo ciclo.

Las imágenes generadas en cada iteración son deprocedadas y almacenadas para poder observar los resultados.

Como alternativa adicional de inicialización se podria utilizar la imagen referencia de estilo e ir ganando progresivamente el contenido a lo largo de las iteraciones. Algunas pruebas con esta alternativa no mostraron buenos resutlados.



In [ ]:
def eval_loss_and_grads(x):
    x = x.reshape((1, img_nrows, img_ncols, 3))
    outs = f_outputs([x])
    loss_value = outs[0]
    if len(outs[1:]) == 1:
        grad_values = outs[1].flatten().astype('float64')
    else:
        grad_values = np.array(outs[1:]).flatten().astype('float64')
    return loss_value, grad_values

# this Evaluator class makes it possible
# to compute loss and gradients in one pass
# while retrieving them via two separate functions,
# "loss" and "grads". This is done because scipy.optimize
# requires separate functions for loss and gradients,
# but computing them separately would be inefficient.

In [ ]:
class Evaluator(object):

    def __init__(self):
        self.loss_value = None
        self.grads_values = None

    def loss(self, x):
        assert self.loss_value is None
        loss_value, grad_values = eval_loss_and_grads(x)
        self.loss_value = loss_value
        self.grad_values = grad_values
        return self.loss_value

    def grads(self, x):
        assert self.loss_value is not None
        grad_values = np.copy(self.grad_values)
        self.loss_value = None
        self.grad_values = None
        return grad_values

# 7) Ejecute la siguiente celda y observe las imágenes de salida en cada iteración.

In [ ]:
evaluator = Evaluator()

# run scipy-based optimization (L-BFGS) over the pixels of the generated image
# so as to minimize the neural style loss
x = preprocess_image(base_image_path)

for i in range(iterations):
    print('Start of iteration', i)
    start_time = time.time()
    x, min_val, info = fmin_l_bfgs_b(evaluator.loss, x.flatten(),
                                     fprime=evaluator.grads, maxfun=20)
    print('Current loss value:', min_val)
    # save current generated image
    img = deprocess_image(x.copy())
    fname = result_prefix / ('output_at_iteration_%d-%d-%d-%d.png' % (i, total_variation_weight*100, style_weight*100, content_weight*100))
    save_img(fname, img)
    end_time = time.time()
    print('Image saved as', fname)
    print('Iteration %d completed in %ds' % (i, end_time - start_time))

# 8) Generar imágenes para distintas combinaciones de pesos de las losses. Explicar las diferencias. (Adjuntar las imágenes generadas como archivos separados.)

Se puede observer como al aumentar el el peso del estilo respectro del de contenido, la imagen comienza a sacrificar un poco de contenido con tal de representar mejor el estilo. A si mimsmo se ve que el aumento de el peso de *total_variation* respecto de los otros pesos, la imagen pierde nitidez progresivamente pero reduce el “overfitting” de algunos puntos aislados.

Las imagenes se encuentran guardadas siguiendo el siguiente código:output_at_iteration_iteración-total_variation_weight*100-style_weight*100-content_weight*100
Adicionalmente se agregan 2 imagenes con las matrices comparativas. OutputComp1 muestra las variaciones de style_weigh y content_weight con el peso de total_variation_weight fijo en 0.1. Por otro lado, OutputComp2 muestra los resultados variando total_variation_weight pero fijando style_weigh y content_weight, ambas en 1.


# 9) Cambiar las imágenes de contenido y estilo por unas elegidas por usted. Adjuntar el resultado.

La imagen (foto) original esta guardada como "contenido", aquella de la cual se captura el estilo esta bajo el nomber "estilo" y la combinada "resultado".